In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib
matplotlib.use('qt5agg')
import matplotlib.pyplot as plt
import time
import numpy as np
import query_db
import composite
import matplotlib.gridspec as gridspec

In [3]:
def make_colorbar(composites):
    params = []
    # for comp in composites:
    # 	params.append(np.average(comp.dm15_array[comp.x1:comp.x2]))
    # norm = matplotlib.colors.Normalize(vmin=np.min(params),vmax=np.max(params))
    norm = matplotlib.colors.Normalize(vmin=1.,vmax=len(composites) + 1.)
    c_m = matplotlib.cm.plasma
    # c_m = matplotlib.cm.winter_r
    s_m = matplotlib.cm.ScalarMappable(cmap=c_m, norm=norm)
    s_m.set_array([])

    return s_m

In [10]:
def comparison_plot(composites):

#     plt.style.use('ggplot')
    colors = [color['color'] for color in list(plt.rcParams['axes.prop_cycle'])]
    h = [3,1,1,1,1,1,1]
    font = {'family' : 'serif',
        'color'  : 'black',
        'weight' : 'bold',
        'size'   : 10,
        }

    gs = gridspec.GridSpec(7, 1, height_ratios=h, hspace = .001)
    fig = plt.figure(num = 1, dpi = 100, figsize = [10,10])
    s_m = make_colorbar(composites)
    lw = 3

    composites, scales = composite.optimize_scales(composites, composites[0], True)

    i = 0
    k = 1
    for comp in composites:
        # param = np.average(comp.dm15_array[comp.x1:comp.x2])
        param = k

        rel_flux = plt.subplot(gs[0])
        plt.setp(rel_flux.get_xticklabels(), visible=False)
        plt.ylabel('Relative Flux', fontdict = font)
        rel_flux.axes.get_yaxis().set_ticks([])
        # plt.plot(comp.wavelength[comp.x1:comp.x2], comp.flux[comp.x1:comp.x2], color = s_m.to_rgba(param))
        plt.plot(comp.wavelength[comp.x1:comp.x2], comp.flux[comp.x1:comp.x2], linewidth = lw, color = s_m.to_rgba(param))
        if len(comp.low_conf) > 0 and len(comp.up_conf) > 0:
            plt.fill_between(comp.wavelength[comp.x1:comp.x2], comp.low_conf[comp.x1:comp.x2], 
                             comp.up_conf[comp.x1:comp.x2], color = s_m.to_rgba(param), alpha = 0.5)
            # plt.fill_between(comp.wavelength[comp.x1:comp.x2], comp.low_conf[comp.x1:comp.x2], 
            # 	             comp.up_conf[comp.x1:comp.x2], color = colors[i%len(colors)], alpha = 0.5)

        var = plt.subplot(gs[1], sharex = rel_flux)
        plt.setp(var.get_xticklabels(), visible=False)
        plt.ylabel('Variance', fontdict = font)
        # plt.plot(comp.wavelength[comp.x1:comp.x2], comp.ivar[comp.x1:comp.x2], color = s_m.to_rgba(param))
        plt.plot(comp.wavelength[comp.x1:comp.x2], comp.ivar[comp.x1:comp.x2], linewidth = lw, color = s_m.to_rgba(param))

        res = plt.subplot(gs[2], sharex = rel_flux)
        plt.setp(res.get_xticklabels(), visible=False)
        plt.ylabel('Residuals', fontdict = font)
        # plt.plot(comp.wavelength[comp.x1:comp.x2], comp.flux[comp.x1:comp.x2] - composites[0].flux[comp.x1:comp.x2], color = s_m.to_rgba(param))
        plt.plot(comp.wavelength[comp.x1:comp.x2], comp.flux[comp.x1:comp.x2] - composites[0].flux[comp.x1:comp.x2], linewidth = lw, color = s_m.to_rgba(param))
        if len(comp.low_conf) > 0 and len(comp.up_conf) > 0:
            low_resid = comp.low_conf[comp.x1:comp.x2] - composites[0].flux[comp.x1:comp.x2]
            up_resid = comp.up_conf[comp.x1:comp.x2] - composites[0].flux[comp.x1:comp.x2]
            plt.fill_between(comp.wavelength[comp.x1:comp.x2], low_resid, up_resid, color = s_m.to_rgba(param), alpha = 0.5)
            # plt.fill_between(comp.wavelength[comp.x1:comp.x2], low_resid, up_resid, color = colors[i%len(colors)], alpha = 0.5)

        spec = plt.subplot(gs[3], sharex = rel_flux)
        plt.setp(spec.get_xticklabels(), visible=False)
        plt.ylabel('Spectra/Bin', fontdict = font)
        # plt.plot(comp.wavelength[comp.x1:comp.x2], comp.spec_bin[comp.x1:comp.x2], color = s_m.to_rgba(param))
        plt.plot(comp.wavelength[comp.x1:comp.x2], comp.spec_bin[comp.x1:comp.x2], linewidth = lw, color = s_m.to_rgba(param))

        phase = plt.subplot(gs[4], sharex = rel_flux)
        plt.setp(phase.get_xticklabels(), visible=False)
        plt.ylabel('Phase [d]', fontdict = font)
        # plt.plot(comp.wavelength[comp.x1:comp.x2], comp.phase_array[comp.x1:comp.x2], color = s_m.to_rgba(param))
        plt.plot(comp.wavelength[comp.x1:comp.x2], comp.phase_array[comp.x1:comp.x2], linewidth = lw, color = s_m.to_rgba(param))

        delt = plt.subplot(gs[5], sharex = rel_flux)
        plt.setp(delt.get_xticklabels(), visible=False)
        plt.ylabel('$\Delta$m$_{15}$', fontdict = font)
        # plt.plot(comp.wavelength[comp.x1:comp.x2], comp.dm15[comp.x1:comp.x2], color = s_m.to_rgba(param))
        plt.plot(comp.wavelength[comp.x1:comp.x2], comp.dm15[comp.x1:comp.x2], linewidth = lw, color = s_m.to_rgba(param))

        plt.subplot(gs[6], sharex = rel_flux)
        plt.ylabel('Redshift', fontdict = font)
        # plt.plot(comp.wavelength[comp.x1:comp.x2], comp.red_array[comp.x1:comp.x2], color = s_m.to_rgba(param))
        plt.plot(comp.wavelength[comp.x1:comp.x2], comp.red_array[comp.x1:comp.x2], linewidth = lw, color = s_m.to_rgba(param))

        i+=1
        k+=1

    plt.xlabel( 'Wavelength ' + "($\mathrm{\AA}$)", fontdict = font)
    # cb = plt.colorbar(s_m, ax = fig.axes)
    # cb.set_label('Phase', fontdict = font)
    # for ax in fig.axes:
    # 	ax.set_axis_bgcolor('white')

    plt.show()

In [5]:
def stacked_plot_max(composites,buff,label_buff):
    font1 = {'family' : 'serif',
        'color'  : 'black',
        'weight' : 'bold',
        'size'   : 20,
        }
    
    font2 = {'family' : 'serif',
        'color'  : 'black',
        'weight' : 'bold',
        'size'   : 10,
        }
    
    fig, ax = plt.subplots(1,1)
    fig.set_size_inches(15, 30, forward = True)
    ax.get_yaxis().set_ticks([])
    ax.xaxis.set_tick_params(length=10, width = 2)
    plt.ylabel('Relative Flux + const.', fontdict = font1, fontsize = 40)
    plt.xlabel( 'Wavelength ' + "($\mathrm{\AA}$)", fontdict = font1, fontsize = 40)
    plt.xticks(fontsize = 20)
    composites, scales = composite.optimize_scales(composites, composites[0], True)

    i = 0
    for comp in composites:
        dm15 = np.average(comp.dm15[comp.x1:comp.x2])
#         ax.plot(comp.wavelength[comp.x1:comp.x2], 2.e15*comp.flux[comp.x1:comp.x2] - 1200*dm15, color = '#3F5D7D', linewidth = 4)
        ax.plot(comp.wavelength[comp.x1:comp.x2], 2.e15*comp.flux[comp.x1:comp.x2] - buff*i, color = '#3F5D7D', linewidth = 4)
#         plt.fill_between(comp.wavelength[comp.x1:comp.x2], 2.e15*comp.low_conf[comp.x1:comp.x2]- buff*i, 2.e15*comp.up_conf[comp.x1:comp.x2] - buff*i, alpha = 0.5)
#         plt.text(9700, comp.flux[comp.x2] - 1200*dm15 + 30, '$\mathbf{\Delta m_{15}}$ = ' + str(round(dm15, 2)), fontdict = font1)
        plt.text(8000, comp.flux[comp.x2] - buff*i +label_buff, '$\mathbf{\Delta m_{15}}$ = ' + str(round(dm15, 2)), fontdict = font2, fontsize = 30)
        i += 1
#     plt.title('Maximum Light Composite Spectra \n for Different Light Curve Shapes', fontdict = font1, fontsize = 40)
    plt.xlim([3000,10500])
#     plt.savefig('../../FOE_figures/dm15_split_max.png', dpi = 300, bbox_inches = 'tight')
    plt.show()

# Hubble Residuals

In [6]:
query_hubble_res = ["SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= -3 and phase <= 3 and Hubble_Res > .1",
                    "SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= -3  and phase <= 3 and Hubble_Res <= -.1"]

In [7]:
comps_hubble_res = query_db.main(len(query_hubble_res), query_hubble_res)

SQL Query: SELECT * from Supernovae inner join Photometry ON Supernovae.SN = Photometry.SN where phase >= -3 and phase <= 3 and Hubble_Res > .1
19 spectra found
151 questionable files currently ignored
bad file removed
1 questionable spectra removed 18 spectra left
0 Peculiar Ias removed 18 spectra left
16 SNs with host corrections
Arrays cleaned
151 questionable files currently ignored
0 spectra with nan ivars removed 16 spectra left
2004gs sn2004gs-20041217.447-ui-corrected.flm 0.44 1.41 None
Host correction...
2005lz sn2005lz-20060101.381-deimos.flm 0.58 1.34 None
Host correction...
2005ms sn2005ms-20060106.400-ui.flm -1.88 None 0.997210431761
Host correction...
2007a sn2007a-20070113.208-br.flm 2.37 0.95 None
Host correction...
1996ab sn1996ab-19960522.37-fast.flm 1.47 1.01 None
Host correction...
1996c sn1996C-19960217.48-fast.flm 1.98 0.93 None
Host correction...
1998bp sn1998bp-19980430.47-fast.flm -2.93 1.79 None
Host correction...
2001ay sn2001ay-20010421.48-fast.flm -1.52 0.6

In [8]:
query_db.set_min_num_spec(comps_hubble_res, 5)
comparison_plot(comps_hubble_res)
stacked_plot_max(comps_hubble_res,5,1)